In [72]:
import tensorflow as tf
from sionna.utils import expand_to_rank
def cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=False):
    real_dtype = tau.dtype

    if len(tau.shape) == 4:
        # Expand dims to broadcast with h. Add the following dimensions:
        #  - number of rx antennas (2)
        #  - number of tx antennas (4)
        tau = tf.expand_dims(tf.expand_dims(tau, axis=2), axis=4)
        # Broadcast is not supported by TF for such high rank tensors.
        # We therefore do part of it manually
        tau = tf.tile(tau, [1, 1, 1, 1, a.shape[4], 1])

    # Add a time samples dimension for broadcasting
    tau = tf.expand_dims(tau, axis=6)

    # Time lags for which to compute the channel taps
    l = tf.range(l_min, l_max+1, dtype=real_dtype)

    # Bring tau and l to broadcastable shapes
    tau = tf.expand_dims(tau, axis=-1)
    l = expand_to_rank(l, tau.shape.rank, axis=0)

    # sinc pulse shaping
    g = tf.experimental.numpy.sinc(l-tau*bandwidth)
    g = tf.complex(g, tf.constant(0., real_dtype))
    a = tf.expand_dims(a, axis=-1)

    # For every tap, sum the sinc-weighted coefficients
    hm = tf.reduce_sum(a*g, axis=-3)

    if normalize:
        # Normalization is performed such that for each batch example and
        # link the energy per block is one.
        # The total energy of a channel response is the sum of the squared
        # norm over the channel taps.
        # Average over block size, RX antennas, and TX antennas
        c = tf.reduce_mean(tf.reduce_sum(tf.square(tf.abs(hm)),
                                         axis=6, keepdims=True),
                           axis=(2,4,5), keepdims=True)
        c = tf.complex(tf.sqrt(c), tf.constant(0., real_dtype))
        hm = tf.math.divide_no_nan(hm, c)

    return hm

In [73]:
import tensorflow as tf
from my_code.mysionna.constants import GLOBAL_SEED_NUMBER

def test_cir_to_time_channel():
    #设置种子
    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # 示例参数
    bandwidth = 20e6  # 20 MHz

    # 生成模拟数据
    batch_size = 1
    num_rx = 1
    num_rx_ant = 1
    num_tx = 1
    num_tx_ant = 1
    num_paths = 5
    num_time_steps = 10

    # 随机生成路径系数（复数）
    a = tf.complex(tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]),
                tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]))

    # 随机生成路径延迟（实数）
    tau = tf.random.uniform([batch_size, num_rx, num_tx, num_paths])

    # 时间抽头范围
    l_min = -5
    l_max = 5

    # 调用函数
    hm = cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=True)

    print(hm)
    print(hm.shape)
    print(hm.dtype)
test_cir_to_time_channel()


tf.Tensor(
[[[[[[[-0.0010297 -0.01116179j  0.04673905+0.02841946j
       -0.12742408-0.13641776j  0.20179068+0.20753935j
       -0.1098018 -0.12750523j -0.07490576-0.06217125j
        0.01046753-0.02048977j  0.01500362+0.04721779j
       -0.09453862-0.1185082j   0.09851032+0.10335578j
       -0.12903695-0.10412582j]
      [ 0.21346247+0.27125633j -0.17321755-0.29813966j
        0.24720478+0.30445552j -0.18566336-0.33407715j
        0.06421079+0.31312802j -0.13005596-0.37832937j
        0.22443095+0.37106553j -0.1978377 -0.28699064j
        0.13127004+0.33030245j -0.26518646-0.3003759j
        0.2386253 +0.30928183j]
      [ 0.12565194+0.05868819j -0.07910819-0.05132939j
        0.11684381+0.10935003j -0.05461621-0.11052839j
        0.11637517+0.2155125j  -0.11721931-0.05677493j
        0.16700381+0.1080543j  -0.1117426 -0.05742691j
        0.04396446+0.05841747j -0.12992199-0.09754615j
        0.09575476+0.08817555j]
      [-0.04500984+0.01795548j  0.02107948+0.07768019j
       -0.0332

In [69]:
import torch
from my_code.mysionna.constants import GLOBAL_SEED_NUMBER
from my_code.mysionna.utils import expand_to_rank
def cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=False):

    # tau = torch.tensor(tau)
    real_dtype = tau.dtype

    if len(tau.shape) == 4:
        # Expand dims to broadcast with h. Add the following dimensions:
        #  - number of rx antennas (2)
        #  - number of tx antennas (4)
        tau = torch.unsqueeze(torch.unsqueeze(tau,dim=2),dim=4)
        # We therefore do part of it manually
        tau = tau.repeat(1,1,1,1,a.shape[4],1)
    
    # Add a time samples dimension for broadcasting
    tau = torch.unsqueeze(tau, dim=6)

    # Time lags for which to compute the channel taps
    l = torch.arange(l_min, l_max+1, dtype=real_dtype)

    # Bring tau and l to broadcastable shapes
    tau = torch.unsqueeze(tau, dim=-1)
    l = expand_to_rank(l, tau.dim(),axis=0)

    # sinc pulse shaping
    g = torch.sinc(l-tau*bandwidth)
    g = torch.complex(g, torch.tensor(0.,dtype=real_dtype))
    a = torch.unsqueeze(a, dim=-1)

    # For every tap, sum the sinc-weighted coefficients
    hm = torch.sum(a*g, dim=-3)

    if normalize:
        # Normalization is performed such that for each batch example and
        # link the energy per block is one.
        # The total energy of a channel response is the sum of the squared
        # norm over the channel taps.
        # Average over block size, RX antennas, and TX antennas
        c = torch.mean(torch.sum(torch.square(torch.abs(hm)),
                                dim=6, 
                                keepdim=True),
                            dim=(2,4,5),
                            keepdim=True
                       )
        c = torch.complex(torch.sqrt(c), torch.tensor(0.,dtype=real_dtype))
        hm = torch.divide(hm, c+1e-10)
    
    return hm

In [71]:
import numpy as np
def test_cir_to_time_channel():
    #设置种子
    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # 示例参数
    bandwidth = 20e6  # 20 MHz

    # 生成模拟数据
    batch_size = 1
    num_rx = 1
    num_rx_ant = 1
    num_tx = 1
    num_tx_ant = 1
    num_paths = 5
    num_time_steps = 10

    # 随机生成路径系数（复数）
    a_tf = tf.complex(tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]),
                tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]))

    # 随机生成路径延迟（实数）
    tau_tf = tf.random.uniform([batch_size, num_rx, num_tx, num_paths])

    #将 TensorFlow 数据转换为 NumPy 数组
    a_np = a_tf.numpy()
    tau_np = tau_tf.numpy()
    # 将 NumPy 数组转换为 PyTorch 张量
    a = torch.from_numpy(a_np)
    tau = torch.from_numpy(tau_np)
    # 时间抽头范围
    l_min = -5
    l_max = 5

    # 调用函数
    hm = cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=True)

    print(hm)
    print(hm.shape)
    print(hm.dtype)
test_cir_to_time_channel()

tensor([[[[[[[-0.0010-0.0112j,  0.0467+0.0284j, -0.1274-0.1364j,
               0.2018+0.2075j, -0.1098-0.1275j, -0.0749-0.0622j,
               0.0105-0.0205j,  0.0150+0.0472j, -0.0945-0.1185j,
               0.0985+0.1033j, -0.1290-0.1041j],
             [ 0.2134+0.2712j, -0.1732-0.2981j,  0.2472+0.3044j,
              -0.1856-0.3340j,  0.0642+0.3131j, -0.1300-0.3783j,
               0.2244+0.3710j, -0.1978-0.2870j,  0.1313+0.3303j,
              -0.2651-0.3003j,  0.2386+0.3092j],
             [ 0.1256+0.0587j, -0.0791-0.0513j,  0.1168+0.1093j,
              -0.0546-0.1105j,  0.1164+0.2155j, -0.1172-0.0568j,
               0.1670+0.1080j, -0.1117-0.0574j,  0.0440+0.0584j,
              -0.1299-0.0975j,  0.0957+0.0882j],
             [-0.0450+0.0180j,  0.0211+0.0777j, -0.0332-0.1732j,
              -0.0047+0.2876j, -0.1032-0.1505j,  0.0583-0.1353j,
              -0.0774+0.0555j,  0.0298-0.0193j,  0.0107-0.1073j,
               0.0392+0.1332j, -0.0231-0.2076j],
             [-0.1637+0.